In [1]:
filepath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC'

In [2]:
import os

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(filepath):
    for file in f:
        if '._' not in file and '.xml' in file and file != 'bookId.xml':
            files.append(os.path.join(r, file))

print(len(files))
print(files[0:5])

3998
['/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35392.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97486.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35386.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/92850.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/36842.xml']


In [3]:
import xmltodict, json

# from anytree import AnyNode, RenderTree
bookTitles = {}
chapterTitles = {}

def chapterWithBook(INchapter):
    for book in bookTitles:
        if INchapter in book:
            return True
    return False
    
def singleChapter(INchapter, INparent):
    _title = INchapter['title']
#     if type(_title) == str and _title.startswith(INparent):
# #         print(INparent, _title)
#         _title = _title[len(INparent):]
#         print(INparent, _title)
#         replaceCount = replaceCount + 1
    
    _from = ""
    if '@from' in INchapter:
        _from = INchapter['@from']
    
    _to = ""
    if '@to' in INchapter:
        _to = INchapter['@to']
        
    _level = ""
    if '@level' in INchapter:
        _level = INchapter['@level']
        
    _n = ""
    if '@n' in INchapter:
        _n = INchapter['@n']
        
#     if INparent not in chapterTitles:
#         chapterTitles[INparent] = 

    if _title not in chapterTitles:
#         chapterTitles.append(_title:INparent)
        if INparent not in bookTitles:
            chapterTitles[_title] = [INparent]
#             AnyNode(id=_title, parent=AnyNode(id=INparent))
    elif INparent not in chapterTitles[_title]:
        if INparent not in bookTitles:
            chapterTitles[_title].append(INparent)
#             AnyNode(id=_title, parent=AnyNode(id=INparent))

#     print(INparent, title, _from, _to, _level, _n)
#     return {title, _from, _to, _level, _n}
    
def section(INarray, INtitle):
    if type(INarray) == dict:
        singleChapter(INarray, INtitle)
        
    elif type(INarray) == list:    
        for ch in INarray:
            section(ch, INtitle)
            if 'section' in ch:
                section(ch['section'], ch['title'])
                
    else:
        print(INarray)
        print("ERROR:", type(INarray))

        
def recursiveSection(INdata, fid):
    book = INdata['title']
    
    if book in bookTitles:
#         print(bookTitles)
#         print("ERROR!", book)
        bookTitles[book].append(fid)
    else:
        bookTitles[book] = [fid]

#     print("BOOK:", title, file)
#     subSection = {}
    if 'section' in INdata:
        section(INdata['section'], book)
        
#     broot = {}
#     broot[fid] = {'hasBookName': title, 'hasSubSection': subSection}
#     return broot

def parseFile(file):
    with open(file) as fp:
        xmlStr = fp.read()
        convertedDict = xmltodict.parse(xmlStr);
        jsonStr = json.dumps(convertedDict, ensure_ascii=False)
        jsonObj = json.loads(jsonStr)

#         print(jsonObj)
        book = jsonObj['book']
    
        rstart = file.rfind('/')
        rend = file.rfind('.')
        recursiveSection(book, file[rstart+1:rend])
#         print(bookTree)
        
# parseFile(files[0])

In [4]:
# root = AnyNode(id="root")
for file in files:
    parseFile(file)
# parseFile(files[0])

In [5]:
# # Remove bookname
# booknamecnt = 0
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue
# #         print(sec)
#         if sec.startswith(tit):
#             if tit not in bookTitles:
#                 # remove the same section names in parent and sub section both
#                 if sec == tit:
#                     chapterTitles[sec].remove(tit)
#                     print(sec, tit)
#             else:
# #                 print(tit)
#                 chapterTitles[sec].remove(tit)

#             booknamecnt += 1
        
#         # remove parent section is book
# #         if tit in bookTitles:
# #             chapterTitles[sec].remove(tit)
# #             booknamecnt += 1
# #             print(sec, tit)
            
# print(booknamecnt)
# # print(len(chapterTitles))

# Write chapters to file

In [6]:
allChapters = []

for key in chapterTitles:
    if key not in allChapters:
        allChapters.append(key)
    for c in chapterTitles[key]:
        if c not in allChapters:
            allChapters.append(c)

print(len(allChapters))

77425


In [7]:
import pandas as pd

resDf = pd.DataFrame.from_dict(allChapters)
# print(resDf)
resDf.sort_values(by=[0],inplace=True)
resDf.head()

,0
33834,(二)河流
58589,□制
53167,□陵墓志
51844,○丘墓
51875,○事紀


In [8]:
outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
write_file_to = '{0}/{1}'.format(outputPath, 'allchapters_20190819_v2.xlsx')
resDf.to_excel(write_file_to, index=True)

# test

In [9]:
dupbooks = {}
for key in bookTitles:
#     print(bookTitles[key])
    if len(bookTitles[key]) > 1:
#         print(key, bookTitles[key])
        dupbooks[key] = bookTitles[key]
print(len(dupbooks))

861


In [10]:
import pandas as pd

resDf = pd.DataFrame.from_dict(dupbooks, orient='index')
# print(resDf)
resDf.sort_index(inplace=True)
resDf.head()

print(len(resDf))

861


In [11]:
outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v2.xlsx')
resDf.to_excel(write_file_to, index=True)

In [12]:
# import pandas as pd

# resDf = pd.DataFrame(chapterTitles, columns=['section'])
# resDf.sort_values(by=['section'], inplace=True)
# resDf.head()

# print(len(resDf))

In [13]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v1.xlsx')
# resDf.to_excel(write_file_to, index=False)

In [14]:
count = 0
for key in chapterTitles:
    print(key, chapterTitles[key])
    count += 1
    if count > 5:
        break

園蔬植物（闕） ['鑛物（闕）']
攝影 ['卷首', '畫']
重修開原縣志序 ['卷首']
章邑侯原序 ['卷首']
明邑侯原序 ['卷首']
胡廣文原序 ['卷首']


In [15]:
# Remove bookname
booknamecnt = 0
for sec in chapterTitles:
#     print(type(sec))
    if sec == None:
        continue
    titles = chapterTitles[sec]
    
    for tit in titles:
        if tit == None:
            continue
#         print(sec)
        if sec.startswith(tit):
            if tit not in bookTitles:
                # remove the same section names in parent and sub section both
                if sec == tit:
                    chapterTitles[sec].remove(tit)
                    print(sec, tit)
            else:
#                 print(tit)
                chapterTitles[sec].remove(tit)

            booknamecnt += 1
        
        # remove parent section is book
#         if tit in bookTitles:
#             chapterTitles[sec].remove(tit)
#             booknamecnt += 1
#             print(sec, tit)
            
print(booknamecnt)
# print(len(chapterTitles))

詩 詩
序 序
藝文 藝文
凡例 凡例
武備志 武備志
學校 學校
兵制 兵制
水利 水利
兵衛 兵衛
官師 官師
進士 進士
目錄 目錄
秩官 秩官
舊序 舊序
教育志 教育志
科舉 科舉
郵舍 郵舍
祭 祭
祀事志 祀事志
舊跋 舊跋
重修邵武縣志序 重修邵武縣志序
沛縣志目次 沛縣志目次
城隍 城隍
都隅 都隅
市巷 市巷
正志校堪記 正志校堪記
南漳縣志集鈔卷之二十二 南漳縣志集鈔卷之二十二
南漳縣志集鈔卷之二十三 南漳縣志集鈔卷之二十三
南漳縣志集鈔卷之二十四 南漳縣志集鈔卷之二十四
襄陽沿革略 襄陽沿革略
息縣志卷之五 息縣志卷之五
366


In [16]:
bookTitles['成安縣志']

['94377', '97288']

In [17]:
# remove parent section is book
for sec in chapterTitles:
#     print(type(sec))
    if sec == None:
        continue
    titles = chapterTitles[sec]
    
    for tit in titles:
        if tit == None:
            continue

        if tit in bookTitles:
            # remove the parent section name which is the same as bookname
            chapterTitles[sec].remove(tit)
            
print(len(chapterTitles))

39049


In [19]:
# 僊居水卷之一
type(chapterTitles)

dict

In [ ]:
import re

for key in chapterTitles:
    titles = chapterTitles[key]
    for t in titles:
        x = re.search(".*卷[首之上下一二三四五六七八九十]*", t)
        if x != None:
            print(t, x)
            chapterTitles[key].remove(t)

In [ ]:
chapterTitles